In [ ]:
# This is necessary to recognize the modules
import os
import sys
import plotly.graph_objects as go


root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


In [ ]:
import pickle

import pandas as pd
path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "backtesting_results_la_maquinita.pickle")

bt_results = pickle.load(open(path, 'rb'))

In [ ]:
import pandas as pd

results_df = pd.DataFrame([bt_result.results for bt_result in bt_results])
results_df["config"] = [bt_result.controller_config.dict() for bt_result in bt_results]
results_df["trading_pair"] = results_df["config"].apply(lambda x: x["trading_pair"])
results_df["experiment_id"] = results_df.index

In [ ]:
import plotly.express as px

# Create a new column with custom hover text
results_df['custom_hover_text'] = results_df.apply(lambda row: f"""
Pair: {row['config']['trading_pair']}
<br>Volume: {row['total_volume']}
<br>PNL: {row['net_pnl_quote']}
<br>Close types: {row['close_types']}

Config:
<br>Id: {row["experiment_id"]}
<br>BB Length: {row['config']['bb_length']}
<br>BB Std: {row['config']['bb_std']}
<br>Take profit: {row['config']['take_profit']}
<br>Stop loss: {row['config']['stop_loss']}
<br>Time limit: {row['config']['time_limit']}
<br>DCA Spreads: {row['config']['dca_spreads']}
<br>DCA Amounts: {row['config']['dca_amounts_pct']}
""", axis=1)

# Create the scatter plot with the custom hover text
fig = px.scatter(
    results_df[(results_df["net_pnl_quote"] > 0) & (results_df["sharpe_ratio"] > 0) & (results_df["total_volume"] > 10000)],
    x="total_volume",
    y="net_pnl_quote",
    size="sharpe_ratio",
    color="trading_pair",  # Color by trading_pair
    hover_data={"custom_hover_text": True},  # Show the custom hover text
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Optional: use a specific color sequence
    title="Net PNL Quote vs. Total Volume by Trading Pair"
)

# Show the figure
fig.show()


In [ ]:
index = 378

bt_result_index = bt_results[index]
print(bt_result_index.controller_config.id)
print(f"DCA Distribution: {bt_result_index.controller_config.dca_amounts_pct} - {bt_result_index.controller_config.dca_spreads}")
print(bt_result_index.get_results_summary())
fig = bt_result_index.get_backtesting_figure()
bb_length = bt_result_index.controller_config.bb_length
bb_std = bt_result_index.controller_config.bb_std
# Add Bollinger Bands
bb_upper = f'BBU_{bb_length}_{bb_std}'
bb_lower = f'BBL_{bb_length}_{bb_std}'
fig.add_trace(go.Scatter(x=bt_result_index.processed_data.index, y=bt_result_index.processed_data[bb_upper],
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         name='BB Upper'), row=1, col=1)
fig.add_trace(go.Scatter(x=bt_result_index.processed_data.index, y=bt_result_index.processed_data[bb_lower],
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         fill='tonexty', fillcolor='rgba(173, 204, 255, 0.1)',
                         name='BB Lower'), row=1, col=1)

In [ ]:
from dotenv import load_dotenv
from services.backend_api_client import BackendAPIClient
load_dotenv()

servers = os.environ.get("SERVERS").split(",")
ts_server = servers[-1]

client = BackendAPIClient(host=ts_server)

In [ ]:
best_configs_index = [61, 322, 277, 8, 117, 337, 240, 64]

best_configs = results_df[results_df["experiment_id"].isin(best_configs_index)]["config"].tolist()

In [ ]:
from enum import Enum

for config in best_configs:
    if "position_mode" in config:
        config["position_mode"] = config["position_mode"].name if isinstance(config["position_mode"], Enum) else config["position_mode"]
    if "take_profit_order_type" in config:
        config["take_profit_order_type"] = config["take_profit_order_type"].name if isinstance(config["take_profit_order_type"], Enum) else config["take_profit_order_type"]
    client.add_controller_config(controller_config=config)
configs_id = [config["id"] + ".yml" for config in best_configs]
configs_id

In [ ]:
client.deploy_script_with_controllers(
    bot_name="la_maquinita",
    controller_configs=configs_id,
    credentials="ts_binance",
    time_to_cash_out=200000,
    max_global_drawdown=70,
    max_controller_drawdown=25
)